In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
train_df = pd.read_csv('/content/drive/MyDrive/machine learning projects/training set/CNN-DailyMail News Text Summarization/train.csv')
train_df.head(2)

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...


In [3]:
train_df = train_df.drop(['id'],axis=1)

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   article     287113 non-null  object
 1   highlights  287113 non-null  object
dtypes: object(2)
memory usage: 4.4+ MB


In [5]:
train_df['article'][0]

"By . Associated Press . PUBLISHED: . 14:11 EST, 25 October 2013 . | . UPDATED: . 15:36 EST, 25 October 2013 . The bishop of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A virus in late September and early October. The state Health Department has issued an advisory of exposure for anyone who attended five churches and took communion. Bishop John Folda (pictured) of the Fargo Catholic Diocese in North Dakota has exposed potentially hundreds of church members in Fargo, Grand Forks and Jamestown to the hepatitis A . State Immunization Program Manager Molly Howell says the risk is low, but officials feel it's important to alert people to the possible exposure. The diocese announced on Monday that Bishop John Folda is taking time off after being diagnosed with hepatitis A. The diocese says he contracted the infection through contaminated food while attending a conference for newly ordained 

In [6]:
train_df['highlights'][0]

'Bishop John Folda, of North Dakota, is taking time off after being diagnosed .\nHe contracted the infection through contaminated food in Italy .\nChurch members in Fargo, Grand Forks and Jamestown could have been exposed .'

In [7]:
import string  
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
sent = []
sent_count = 0
for sentence in train_df['article']:
    sent_count += 1
    sentence_tokens = word_tokenize(sentence)
    sent.append(sentence_tokens)
    if sent_count >= 1:
        break

In [9]:
print(sent)

[['By', '.', 'Associated', 'Press', '.', 'PUBLISHED', ':', '.', '14:11', 'EST', ',', '25', 'October', '2013', '.', '|', '.', 'UPDATED', ':', '.', '15:36', 'EST', ',', '25', 'October', '2013', '.', 'The', 'bishop', 'of', 'the', 'Fargo', 'Catholic', 'Diocese', 'in', 'North', 'Dakota', 'has', 'exposed', 'potentially', 'hundreds', 'of', 'church', 'members', 'in', 'Fargo', ',', 'Grand', 'Forks', 'and', 'Jamestown', 'to', 'the', 'hepatitis', 'A', 'virus', 'in', 'late', 'September', 'and', 'early', 'October', '.', 'The', 'state', 'Health', 'Department', 'has', 'issued', 'an', 'advisory', 'of', 'exposure', 'for', 'anyone', 'who', 'attended', 'five', 'churches', 'and', 'took', 'communion', '.', 'Bishop', 'John', 'Folda', '(', 'pictured', ')', 'of', 'the', 'Fargo', 'Catholic', 'Diocese', 'in', 'North', 'Dakota', 'has', 'exposed', 'potentially', 'hundreds', 'of', 'church', 'members', 'in', 'Fargo', ',', 'Grand', 'Forks', 'and', 'Jamestown', 'to', 'the', 'hepatitis', 'A', '.', 'State', 'Immunizati

In [10]:
stopwords_english = stopwords.words('english') 
punctuation = string.punctuation
punctuation = punctuation.replace('.','')


formatted_sent = []
for sentence in sent:
    formatted_words = []
    for word in sentence:
        if word not in stopwords_english and word not in punctuation:  #Removes word with less than 2 characters, present in english stop words or is a punctuation
            formatted_words.append(word.lower())
    formatted_sent.append(formatted_words)

In [11]:
print(formatted_sent)

[['by', '.', 'associated', 'press', '.', 'published', '.', '14:11', 'est', '25', 'october', '2013', '.', '.', 'updated', '.', '15:36', 'est', '25', 'october', '2013', '.', 'the', 'bishop', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundreds', 'church', 'members', 'fargo', 'grand', 'forks', 'jamestown', 'hepatitis', 'a', 'virus', 'late', 'september', 'early', 'october', '.', 'the', 'state', 'health', 'department', 'issued', 'advisory', 'exposure', 'anyone', 'attended', 'five', 'churches', 'took', 'communion', '.', 'bishop', 'john', 'folda', 'pictured', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundreds', 'church', 'members', 'fargo', 'grand', 'forks', 'jamestown', 'hepatitis', 'a', '.', 'state', 'immunization', 'program', 'manager', 'molly', 'howell', 'says', 'risk', 'low', 'officials', 'feel', "'s", 'important', 'alert', 'people', 'possible', 'exposure', '.', 'the', 'diocese', 'announced', 'monday', 'bishop', 'john'

In [12]:
import re
otherwords = ['by','associated','press','published','est','october','updated']


formatted_sent_two = []
for sentence in formatted_sent:
    formatted_words = []
    for word in sentence:
        if word not in otherwords and not re.search('\d{2}:\d{2}',word) and not re.search('\d{2,4}',word):  #Removes time, year and few other irrelevant terms.
            formatted_words.append(word)
    formatted_sent_two.append(formatted_words)

In [13]:
print(formatted_sent_two)

[['.', '.', '.', '.', '.', '.', '.', 'the', 'bishop', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundreds', 'church', 'members', 'fargo', 'grand', 'forks', 'jamestown', 'hepatitis', 'a', 'virus', 'late', 'september', 'early', '.', 'the', 'state', 'health', 'department', 'issued', 'advisory', 'exposure', 'anyone', 'attended', 'five', 'churches', 'took', 'communion', '.', 'bishop', 'john', 'folda', 'pictured', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundreds', 'church', 'members', 'fargo', 'grand', 'forks', 'jamestown', 'hepatitis', 'a', '.', 'state', 'immunization', 'program', 'manager', 'molly', 'howell', 'says', 'risk', 'low', 'officials', 'feel', "'s", 'important', 'alert', 'people', 'possible', 'exposure', '.', 'the', 'diocese', 'announced', 'monday', 'bishop', 'john', 'folda', 'taking', 'time', 'diagnosed', 'hepatitis', 'a', '.', 'the', 'diocese', 'says', 'contracted', 'infection', 'contaminated', 'food', 'att

In [14]:
lemma_sent = []

lemma = WordNetLemmatizer()

for sentence in formatted_sent_two:
    lemma_words = []
    for word in sentence:
        lemma_word = lemma.lemmatize(word)
        lemma_words.append(lemma_word)
    lemma_sent.append(lemma_words)

In [15]:
print(lemma_sent)

[['.', '.', '.', '.', '.', '.', '.', 'the', 'bishop', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundred', 'church', 'member', 'fargo', 'grand', 'fork', 'jamestown', 'hepatitis', 'a', 'virus', 'late', 'september', 'early', '.', 'the', 'state', 'health', 'department', 'issued', 'advisory', 'exposure', 'anyone', 'attended', 'five', 'church', 'took', 'communion', '.', 'bishop', 'john', 'folda', 'pictured', 'fargo', 'catholic', 'diocese', 'north', 'dakota', 'exposed', 'potentially', 'hundred', 'church', 'member', 'fargo', 'grand', 'fork', 'jamestown', 'hepatitis', 'a', '.', 'state', 'immunization', 'program', 'manager', 'molly', 'howell', 'say', 'risk', 'low', 'official', 'feel', "'s", 'important', 'alert', 'people', 'possible', 'exposure', '.', 'the', 'diocese', 'announced', 'monday', 'bishop', 'john', 'folda', 'taking', 'time', 'diagnosed', 'hepatitis', 'a', '.', 'the', 'diocese', 'say', 'contracted', 'infection', 'contaminated', 'food', 'attending', 'c

In [16]:
final_sentence = []

for sentence in lemma_sent:
    words = ' '.join([str(word) for word in sentence])
    final_sentence.append(words)

In [17]:
print(final_sentence)

[". . . . . . . the bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early . the state health department issued advisory exposure anyone attended five church took communion . bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a . state immunization program manager molly howell say risk low official feel 's important alert people possible exposure . the diocese announced monday bishop john folda taking time diagnosed hepatitis a . the diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month . symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort . fargo catholic diocese north dakota pictured bishop located ."]


In [18]:
sentence = str(final_sentence[0][14:])
print(sentence)

the bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early . the state health department issued advisory exposure anyone attended five church took communion . bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a . state immunization program manager molly howell say risk low official feel 's important alert people possible exposure . the diocese announced monday bishop john folda taking time diagnosed hepatitis a . the diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month . symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort . fargo catholic diocese north dakota pictured bishop located .


In [19]:
sentence = sentence.replace('\'s','')
print(sentence)

the bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early . the state health department issued advisory exposure anyone attended five church took communion . bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a . state immunization program manager molly howell say risk low official feel  important alert people possible exposure . the diocese announced monday bishop john folda taking time diagnosed hepatitis a . the diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month . symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort . fargo catholic diocese north dakota pictured bishop located .


In [20]:
del sent
del formatted_sent
del formatted_sent_two
del lemma_sent

In [22]:
word_count = {}
for word in word_tokenize(sentence):
    if word not in ['.']:
        if word in word_count.keys():
            word_count[word] +=1
        else:
            word_count[word] = 1

In [23]:
print(word_count)

{'the': 4, 'bishop': 5, 'fargo': 5, 'catholic': 3, 'diocese': 5, 'north': 3, 'dakota': 3, 'exposed': 2, 'potentially': 2, 'hundred': 2, 'church': 3, 'member': 2, 'grand': 2, 'fork': 2, 'jamestown': 2, 'hepatitis': 4, 'a': 4, 'virus': 1, 'late': 1, 'september': 1, 'early': 1, 'state': 2, 'health': 1, 'department': 1, 'issued': 1, 'advisory': 1, 'exposure': 2, 'anyone': 1, 'attended': 1, 'five': 1, 'took': 1, 'communion': 1, 'john': 2, 'folda': 2, 'pictured': 2, 'immunization': 1, 'program': 1, 'manager': 1, 'molly': 1, 'howell': 1, 'say': 2, 'risk': 1, 'low': 1, 'official': 1, 'feel': 1, 'important': 1, 'alert': 1, 'people': 1, 'possible': 1, 'announced': 1, 'monday': 1, 'taking': 1, 'time': 1, 'diagnosed': 1, 'contracted': 1, 'infection': 1, 'contaminated': 1, 'food': 1, 'attending': 1, 'conference': 1, 'newly': 1, 'ordained': 1, 'italy': 1, 'last': 1, 'month': 1, 'symptom': 1, 'include': 1, 'fever': 1, 'tiredness': 1, 'loss': 1, 'appetite': 1, 'nausea': 1, 'abdominal': 1, 'discomfort'

In [24]:
fin_max = max(word_count, key=word_count.get)
print("Maximum value word:",fin_max)

Maximum value word: bishop


In [25]:
max_val = word_count[fin_max]
print("Maximum value:",max_val)

Maximum value: 5


In [26]:
for key,value in word_count.items():
    word_count[key] = word_count[key]/max_val

In [27]:
print(word_count)

{'the': 0.8, 'bishop': 1.0, 'fargo': 1.0, 'catholic': 0.6, 'diocese': 1.0, 'north': 0.6, 'dakota': 0.6, 'exposed': 0.4, 'potentially': 0.4, 'hundred': 0.4, 'church': 0.6, 'member': 0.4, 'grand': 0.4, 'fork': 0.4, 'jamestown': 0.4, 'hepatitis': 0.8, 'a': 0.8, 'virus': 0.2, 'late': 0.2, 'september': 0.2, 'early': 0.2, 'state': 0.4, 'health': 0.2, 'department': 0.2, 'issued': 0.2, 'advisory': 0.2, 'exposure': 0.4, 'anyone': 0.2, 'attended': 0.2, 'five': 0.2, 'took': 0.2, 'communion': 0.2, 'john': 0.4, 'folda': 0.4, 'pictured': 0.4, 'immunization': 0.2, 'program': 0.2, 'manager': 0.2, 'molly': 0.2, 'howell': 0.2, 'say': 0.4, 'risk': 0.2, 'low': 0.2, 'official': 0.2, 'feel': 0.2, 'important': 0.2, 'alert': 0.2, 'people': 0.2, 'possible': 0.2, 'announced': 0.2, 'monday': 0.2, 'taking': 0.2, 'time': 0.2, 'diagnosed': 0.2, 'contracted': 0.2, 'infection': 0.2, 'contaminated': 0.2, 'food': 0.2, 'attending': 0.2, 'conference': 0.2, 'newly': 0.2, 'ordained': 0.2, 'italy': 0.2, 'last': 0.2, 'month'

In [28]:
for sentences in sent_tokenize(sentence):
    print(sentences)

the bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early .
the state health department issued advisory exposure anyone attended five church took communion .
bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a .
state immunization program manager molly howell say risk low official feel  important alert people possible exposure .
the diocese announced monday bishop john folda taking time diagnosed hepatitis a .
the diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month .
symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort .
fargo catholic diocese north dakota pictured bishop located .


In [29]:
sentence_score = {}

for sentences in sent_tokenize(sentence):
    sentence_score[sentences] = 0
    for word in sentences:
        if word in word_count.keys():
            sentence_score[sentences]+=word_count[word]

In [30]:
print(sentence_score)

{'the bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early .': 9.6, 'the state health department issued advisory exposure anyone attended five church took communion .': 4.8, 'bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a .': 8.799999999999999, 'state immunization program manager molly howell say risk low official feel  important alert people possible exposure .': 7.199999999999999, 'the diocese announced monday bishop john folda taking time diagnosed hepatitis a .': 5.6, 'the diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month .': 6.3999999999999995, 'symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort .': 5.6, 'fargo catholic diocese north dakota pictured bishop located .': 4.0}


In [31]:
from heapq import nlargest
len(sentence_score) *0.2

1.6

In [32]:
summary=nlargest(n=2,iterable=sentence_score,key=sentence_score.get) 
print(summary)

['the bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early .', 'bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a .']


In [33]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 6.6 MB/s 
     |████████████████████████████████| 596 kB 10.0 MB/s 
     |████████████████████████████████| 6.6 MB 38.6 MB/s 
     |████████████████████████████████| 86 kB 5.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [34]:
import transformers
from transformers import pipeline

In [35]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [36]:
sentence_list = []
for sentences in sent_tokenize(sentence):
    sentence_list.append(sentences)

In [37]:
print(sentence_list)

['the bishop fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a virus late september early .', 'the state health department issued advisory exposure anyone attended five church took communion .', 'bishop john folda pictured fargo catholic diocese north dakota exposed potentially hundred church member fargo grand fork jamestown hepatitis a .', 'state immunization program manager molly howell say risk low official feel  important alert people possible exposure .', 'the diocese announced monday bishop john folda taking time diagnosed hepatitis a .', 'the diocese say contracted infection contaminated food attending conference newly ordained bishop italy last month .', 'symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort .', 'fargo catholic diocese north dakota pictured bishop located .']


In [38]:
summary = summarizer(sentence_list,min_length = 10,max_length = 15)

In [39]:
print(summary)

[{'summary_text': ' the bishop fargo catholic diocese north dakota'}, {'summary_text': ' The state health department issued advisory exposure anyone attended five church took'}, {'summary_text': ' bishop john folda exposed potentially hundred church members to hepatitis a'}, {'summary_text': ' state immunization program manager molly howell say risk low'}, {'summary_text': ' bishop john folda taking time diagnosed with hepatitis a . Bishop'}, {'summary_text': ' the diocese say contracted infection contaminated food attending conference newly ordained'}, {'summary_text': ' symptom hepatitis a include fever tiredness loss appetite nausea abdominal discomfort'}, {'summary_text': ' Fargo catholic diocese north dakota pictured bishop located'}]
